### Tratamento de dados: junção, combinação e reformatação

### Indexação hierárquica
A *indexação hierárquica* é um recurso importante do pandas; ela
permite ter vários níveis de índices (dois ou mais) em um eixo. De
forma, até certo ponto, abstrata, ela oferece uma maneira de
trabalhar com dados de dimensões mais altas em um formato de
dimensões menores. Vamos começar com um exemplo simples; crie
uma Series com uma lista de listas (ou de arrays) como índice:

In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]]
                 )

In [3]:
data

a  1    1.153711
   2    1.348400
   3    0.597021
b  1   -1.519302
   3   -0.145111
c  1   -0.741002
   2   -0.442448
d  2   -0.044210
   3    1.099114
dtype: float64

O que você está vendo é uma visão elegante de uma Series com
um *MultiIndex* como índice. As “lacunas” na exibição do índice
significam “utilize o rótulo imediatamente anterior”:

In [4]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

Com um objeto hierarquicamente indexado, a chamada indexação
*parcial* é possível, permitindo selecionar subconjuntos dos dados de
forma concisa:

In [5]:
data['b']

1   -1.519302
3   -0.145111
dtype: float64

In [6]:
data['b': 'c']

b  1   -1.519302
   3   -0.145111
c  1   -0.741002
   2   -0.442448
dtype: float64

In [7]:
data.loc[['b','c']]

b  1   -1.519302
   3   -0.145111
c  1   -0.741002
   2   -0.442448
dtype: float64

A seleção é até mesmo possível a partir de um nível “mais interno”:

In [8]:
data.loc[:, 2]

a    1.348400
c   -0.442448
d   -0.044210
dtype: float64

A indexação hierárquica desempenha um papel importante na
reformatação dos dados e nas operações baseadas em grupos,
como compor uma tabela pivô. Por exemplo, poderíamos
reorganizar os dados em um DataFrame usando o seu método
*unstack*:

In [9]:
data.unstack()

,1,2,3
a,1.153711,1.348400,0.597021
b,-1.519302,NaN,-0.145111
c,-0.741002,-0.442448,NaN
d,NaN,-0.044210,1.099114


A operação inversa de *unstack* é *stack*:

In [10]:
data.unstack().stack()

a  1    1.153711
   2    1.348400
   3    0.597021
b  1   -1.519302
   3   -0.145111
c  1   -0.741002
   2   -0.442448
d  2   -0.044210
   3    1.099114
dtype: float64

Em um DataFrame, qualquer eixo pode ter um índice hierárquico:

In [11]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [12]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

OBS: *Tome cuidado para distinguir os nomes dos índices 'state' e 'color' dos rótulos das linhas.*


Com uma indexação parcial de colunas, você poderá, de modo
semelhante, selecionar grupos de colunas:

In [13]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

Um *MultiIndex* pode ser criado de modo independente e então ser
reutilizado; as colunas no DataFrame anterior com nomes para os
níveis poderiam ter sido criadas assim:

In [14]:
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                          names=['state', 'color'])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

### Reorganizando e ordenando níveis

Ocasionalmente precisaremos reorganizar a ordem dos níveis em
um eixo ou ordenar os dados de acordo com os valores em um nível
específico. *swaplevel* aceita dois números ou nomes de níveis e
devolve um novo objeto com os níveis trocados (os dados, porém,
permanecem inalterados):

In [15]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

*sort_index*, por outro lado, ordena os dados usando os valores de um
só nível. Ao trocar níveis, não é incomum usar *sort_index* também
para que o resultado seja ordenado em ordem lexicográfica de
acordo com o nível indicado:

In [16]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [17]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

OBS: *O desempenho da seleção de dados será muito melhor em objetos
indexados hierarquicamente se o índice estiver ordenado em ordem
lexicográfica, começando pelo nível mais externo – isto é, o resultado da
chamada a sort_index(level=0) ou sort_index().*

### Estatísticas de resumo por nível

Muitas estatísticas descritivas ou de resumo em DataFrame e em
Series têm uma opção *level*; com ela, podemos especificar o nível de
acordo com o qual queremos fazer uma agregação, em um eixo em
particular. Considere o DataFrame anterior; podemos fazer uma
agregação por nível, seja nas linhas ou nas colunas, da seguinte
maneira:

In [18]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [19]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

In [20]:
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

Internamente, o recurso groupby do pandas foi utilizado nesse caso;

### Indexando com as colunas de um DataFrame
Não é incomum querer usar uma ou mais colunas de um DataFrame
como índice de linha; de modo alternativo, talvez você queira mover
o índice das linhas para as colunas do DataFrame. Eis um
DataFrame de exemplo:

In [21]:
frame = pd.DataFrame({'a': range(7), 
                      'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


A função *set_index* de DataFrame criará um novo DataFrame usando
uma ou mais de suas colunas como índice:

In [22]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

Por padrão, as colunas são removidas do DataFrame, embora
possam ser mantidas:

In [23]:
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

*reset_index*, por outro lado, faz o inverso de *set_index*; os níveis dos
índices hierárquicos são passados para as colunas:

In [24]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


### Combinando e mesclando conjuntos de dados
Os dados contidos em objetos do pandas podem ser combinados de
várias maneiras:
 - *pandas.merge* conecta linhas em DataFrames com base em uma
ou mais chaves. Essa operação será conhecida dos usuários de
SQL ou de outros bancos de dados relacionais, pois ela
implementa as operações de *junção* (join) dos bancos de dados.
 - *pandas.concat* concatena ou “empilha” objetos ao longo de um
eixo.
 - O método de instância *combine_first* permite combinar dados que
se sobrepõem a fim de preencher valores ausentes em um objeto
com valores de outro objeto.

### Junções no DataFrame no estilo de bancos de dados
Operações de *merge* (mescla) ou de *junção* (join) combinam conjuntos
de dados associando linhas por meio de uma ou mais *chaves*. Essas
operações são essenciais em bancos de dados relacionais (por
exemplo, naqueles baseados em SQL). A função *merge* do pandas é
o ponto de entrada principal para usar esses algoritmos em seus
dados.

Vamos começar com um exemplo simples:

In [25]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                   'data1': range(7)})

df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})

In [26]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [28]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


O exemplo a seguir é de uma junção de *muitos-para-um* (many-to-
one); os dados em *df1* têm várias linhas de rótulos *a* e *b*, enquanto
*df2* tem apenas uma linha para cada valor na coluna *key*. Se merge for
chamado nesses objetos, teremos o seguinte:

In [29]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


Observe que eu não especifiquei a coluna para fazer a junção. Se
essa informação não for especificada, *merge* utilizará como chaves
os nomes das colunas que se sobrepõem. No entanto, especificá-la
explicitamente é uma boa prática:

In [30]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


Se os nomes das colunas forem diferentes em cada objeto, você poderá especificá-las separadamente:

In [31]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'], 'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'], 'data2': range(3)})

In [32]:
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


Talvez você tenha percebido que os valores *'c'* e *'d'* e os dados
associados estão ausentes no resultado. Por padrão, merge executa
uma junção do tipo *'inner'* (interna); as chaves no resultado são a
intersecção, ou o conjunto comum que se encontra nas duas
tabelas. Outras opções possíveis são: *‘left'*, *'right'* e *'outer'*. A junção
externa (outer join) efetua a união das chaves, combinando o efeito
da aplicação das junções tanto à esquerda quanto à direita:

In [33]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


### Tabela – Diferentes tipos de junção com o argumento how
Opção | Comportamento
------|------------------------
**'inner'** | Utiliza somente as combinações de chaves observadas nas duas tabelas
**'left'** | Utiliza todas as combinações de chaves encontradas na tabela à esquerda
**'right'** | Utiliza todas as combinações de chaves encontradas na tabela à direita
**'outer'** | Utiliza todas as combinações de chaves observadas nas duas tabelas em conjunto

Merges de *muitos-para-muitos* (many-to-many) têm um comportamento
bem definido, embora não seja necessariamente intuitivo. Eis um
exemplo:

In [34]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'], 'data1': range(6)})
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'], 'data2': range(5)})

In [35]:
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [36]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [37]:
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


Junções de muitos para muitos formam o produto cartesiano das
linhas. Como havia três linhas *'b'* no DataFrame da esquerda e duas
no da direita, há seis linhas *'b'* no resultado. O método de junção
afeta somente os valores de chaves distintos que aparecem no
resultado:

In [38]:
pd.merge(df1, df2, how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


Para um merge com várias chaves, passe uma lista de nomes de
coluna:

In [39]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})

right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})


In [40]:
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


Para determinar quais combinações de chaves aparecerão no
resultado conforme a escolha do método de merge, pense nas
várias chaves como compondo um array de tuplas a ser usado
como uma única chave de junção (apesar de isso não ser realmente
implementado dessa forma).

*Ao fazer a junção de colunas-sobre-colunas, os índices dos objetos
DataFrame especificados serão descartados.*

Uma última questão a ser considerada em operações de merge é o
tratamento dos nomes de coluna que se sobrepõem. Embora seja
possível tratar a sobreposição manualmente (veja a seção anterior
sobre renomear rótulos de eixos), *merge* tem uma opção *suffixes* para
especificar strings a serem concatenadas nos nomes que se
sobrepõem, nos objetos DataFrame à esquerda e à direita:



In [41]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [42]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


### Tabela – Argumentos da função merge

Argumento | Descrição
----------|-------------------
**left** | Dataframe à esquerda para o merge
**right** | Dataframe à direita para o merge
**how** | Uma opção entre 'inner', 'outer', 'left' ou 'right'; o default é 'inner
**on** | Nomes de coluna para a junção. Deve ser encontrado nos dois
|objetos DataFrame. Se não for especificado e nenhuma outra chave
|de junção for definida, usará a intersecção entre os nomes de
|colunas em left e em right como as chaves da junção
**left_on** | Colunas no DataFrame left a serem usadas como chaves de junção
**right_on** | Análogo ao left_on para o DataFrame left
**left_index** | Utiliza o índice de linha em left como sua chave de junção (ou
|chaves, se for um MultiIndex)
**right_index** | Análogo a left_index
**sort** | Ordena os dados do merge em ordem lexicográfica, de acordo com
|as chaves de junção; o default é True (desative-o para ter um
|desempenho melhor em alguns casos, quando os conjuntos de
|dados forem grandes)
**suffixes** | Tupla de valores de strings a serem concatenados aos nomes de
|colunas em caso de sobreposição; o default é ('_x', '_y') (por
|exemplo, se houver ‘data’ nos dois objetos DataFrame, esses
|aparecerão como 'data_x' e 'data_y' no resultado)
**copy** | Se for False, evita copiar dados para a estrutura de dados
|resultante em alguns casos excepcionais; por padrão, sempre copia
|indicator Adiciona uma coluna especial _merge que informa a origem de
|cada linha; os valores serão 'left_only', 'right_only' ou 'both'
|conforme a origem dos dados da junção em cada linha

### Fazendo merge com base no índice
Em alguns casos, a(s) chave(s) do merge em um DataFrame serão
encontradas em seu índice. Nessa situação, você poderá passar
*left_index=True* ou *right_index=True* (ou ambos) para informar que o
índice deverá ser usado como a chave do merge:

In [44]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'], 'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])

In [45]:
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [46]:
right1

,group_val
a,3.5
b,7.0


In [47]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


Como o método default de merge consiste em fazer uma
intersecção das chaves de junção, você poderá, de modo
alternativo, compor a união delas fazendo uma junção externa:

In [48]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


Com dados hierarquicamente indexados, a situação é mais
complicada, pois fazer uma junção com base no índice é
implicitamente um merge de várias chaves:

In [52]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})

righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                       index=[['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                              [2001, 2000, 2000, 2000, 2001, 2002]],
                       columns=['event1', 'event2'])


In [53]:
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [54]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

Nesse caso, você deverá especificar várias colunas com base nas
quais o merge será feito, na forma de uma lista (observe o
tratamento de valores de índice duplicados com *how='outer'*):

In [55]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


In [56]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True, how='outer')

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4.0,5.0
0,Ohio,2000,0.0,6.0,7.0
1,Ohio,2001,1.0,8.0,9.0
2,Ohio,2002,2.0,10.0,11.0
3,Nevada,2001,3.0,0.0,1.0
4,Nevada,2002,4.0,NaN,NaN
4,Nevada,2000,NaN,2.0,3.0


Usar os índices dos dois lados do merge também é possível:

In [57]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])

In [58]:
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [59]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [60]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


O DataFrame tem uma instância conveniente de *join* para fazer o
merge pelo índice. Ela também pode ser usada para combinar
vários objetos DataFrame com índices iguais ou semelhantes,
porém com colunas que não se sobrepõem. No exemplo anterior,
poderíamos ter escrito:

In [61]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


Em parte por questões de legado (isto é, versões bem antigas do
pandas), o método *join* de DataFrame realiza uma junção à
esquerda nas chaves de junção, preservando exatamente o índice
de linhas do frame à esquerda. Ele também aceita a junção do
índice do DataFrame recebido em uma das colunas do DataFrame
que faz a chamada:

In [62]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


Por fim, para merges simples de índice-sobre-índice, podemos
passar uma lista de DataFrames para *join* como uma alternativa ao
uso da função *concat*, mais genérica, que será descrita na próxima
seção:

In [63]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12], [16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])

In [64]:
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [65]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


In [66]:
left2.join([right2, another], how='outer')

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0
b,NaN,NaN,7.0,8.0,NaN,NaN
d,NaN,NaN,11.0,12.0,NaN,NaN
f,NaN,NaN,NaN,NaN,16.0,17.0


### Concatenando ao longo de um eixo
Outro tipo de operação de combinação de dados é chamado, de
forma indistinta, de concatenação, vinculação (binding) ou
empilhamento (stacking). A função *concatenate* do NumPy é capaz de
fazer isso com arrays NumPy:

In [67]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [68]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

No contexto de objetos do pandas, como Series e DataFrame, ter
eixos com rótulos permite generalizar melhor a concatenação de
arrays. Em particular, há um número adicional de aspectos nos
quais devemos pensar:

- Se os objetos estiverem indexados de modo diferente nos outros
eixos, devemos combinar os elementos distintos nesses eixos ou
usar somente os valores compartilhados (a intersecção)?

- As porções de dados concatenadas devem ser identificáveis no
objeto resultante?

- O “eixo de concatenação” contém dados que devam ser
preservados? Em muitos casos, será melhor que os rótulos
inteiros default em um DataFrame sejam descartados durante a
concatenação.

A função *concat* do pandas oferece uma forma consistente de tratar
cada um desses aspectos. Apresentarei uma série de exemplos
para mostrar o seu funcionamento. Suponha que tenhamos trêsSeries sem sobreposição de índices:

In [69]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

Chamar *concat* com esses objetos em uma lista reúne os valores e
os índices:

In [70]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

Por padrão, *concat* atua em *axis=0*, gerando outra Series. Se você
passar *axis=1*, o resultado será um DataFrame (*axis=1* são as
colunas):

In [71]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


Nesse caso, não há sobreposição no outro eixo que, como podemos
ver, é a união ordenada (a junção *'outer'*) dos índices. Por outro lado,
podemos fazer a sua intersecção passando *join='inner'*:

In [73]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [74]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [75]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


Nesse último exemplo, os rótulos *'f'* e *'g'* desapareceram por causa
da opção *join='inner'*.

Podemos até mesmo especificar os eixos a serem usados nos
demais eixos com *join_axes*:

In [78]:
pd.concat([s1, s4], axis=1, names=['a', 'c', 'b', 'e'])

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


Um possível problema é o fato de as partes concatenadas não
serem identificáveis no resultado. Suponha que quiséssemos criar
um índice hierárquico no eixo da concatenação. Para isso, utilize o
argumento *keys*:

In [81]:
result = pd.concat([s1, s2, s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    c    2
       d    3
       e    4
three  f    5
       g    6
dtype: int64

In [82]:
result.unstack()

,a,b,c,d,e,f,g
one,0.0,1.0,NaN,NaN,NaN,NaN,NaN
two,NaN,NaN,2.0,3.0,4.0,NaN,NaN
three,NaN,NaN,NaN,NaN,NaN,5.0,6.0


No caso da combinação de Series ao longo de *axis=1*, as *keys*
passam a ser os cabeçalhos das colunas do DataFrame:

In [83]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


A mesma lógica se estende aos objetos DataFrame:

In [84]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'], columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'], columns=['three', 'four'])

In [85]:
df1

,one,two
a,0,1
b,2,3
c,4,5


In [87]:
df2

,three,four
a,5,6
c,7,8


In [88]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

Se você passar um dicionário de objetos no lugar de uma lista, as
chaves do dicionário serão usadas para a opção *keys*:

In [89]:
pd.concat({'level1': df1, 'level2': df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

Há argumentos adicionais que determinam o modo como o índice
hierárquico é criado (veja a Tabela abaixo). Por exemplo, podemos
nomear os níveis criados no eixo com o argumento *names*:

In [90]:
pd.concat([df1, df2], axis=1, keys=['level1', 'leve2'], names=['upper', 'lower'])

upper level1     leve2     
lower    one two three four
a          0   1   5.0  6.0
b          2   3   NaN  NaN
c          4   5   7.0  8.0

Uma última consideração diz respeito aos DataFrames em que o
índice das linhas não contém nenhum dado relevante:

In [91]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])

In [92]:
df1

,a,b,c,d
0,0.534056,-0.185560,2.242354,1.419193
1,-0.187314,0.146604,-1.352949,0.792824
2,1.843292,-1.954932,-0.786483,-1.492686


In [93]:
df2

,b,d,a
0,1.230115,0.706629,0.053505
1,-1.263929,1.269936,1.530986


Nesse caso, podemos passar *ignore_index=True*:

In [94]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,0.534056,-0.185560,2.242354,1.419193
1,-0.187314,0.146604,-1.352949,0.792824
2,1.843292,-1.954932,-0.786483,-1.492686
3,0.053505,1.230115,NaN,0.706629
4,1.530986,-1.263929,NaN,1.269936


### Tabela – Argumentos da função concat
Argumento | Descrição
----------|--------------
**objs** | Lista ou dicionário de objetos do pandas a serem concatenados;
|é o único argumento obrigatório
**axis** | Eixo no qual será feita a concatenação; o default é 0 (nas linhas)
**join** | Pode ser 'inner' ou 'outer' (o default é 'outer'); indica se será feita
|a intersecção (inner) ou a união (outer) dos índices ao longo dos
|outros eixos
**join_axes** | Índices específicos a serem usados para os outros n–1 eixos, em
|vez de executar a lógica de união/intersecção
**keys** | Valores a serem associados aos objetos sendo concatenados,
|compondo um índice hierárquico ao longo do eixo de
|concatenação; pode ser uma lista ou um array de valores
|arbitrários, um array de tuplas ou uma lista de arrays (se arrays
|de vários níveis forem especificados em levels)
**levels** | Índices específicos a serem usados como nível ou níveis de
|índices hierárquicos se chaves forem especificadas
**names** | Nomes para os níveis hierárquicos criados se keys e/ou levels
|forem especificados
**verify_integrity** | Verifica o novo eixo no objeto concatenado em busca de
|duplicatas e lança uma exceção em caso afirmativo; por padrão
|(False), permite duplicatas
**ignore_index** | Não preserva os índices ao longo do axis de concatenação,
|gerando um novo índice range(total_length) em seu lugar



### Combinando dados com sobreposição

Há outra situação de combinação de dados que não pode ser
expressa nem com uma operação de merge nem como de
concatenação. Podemos ter dois conjuntos de dados cujos índices
se sobreponham de forma total ou parcial. Como um exemplo
motivador, considere a função *where* do NumPy, que executa o
equivalente a uma expressão if-else, porém orientado a arrays:

In [96]:
a = pd.Series([np.nan, 2.5, 0.0, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series([0., np.nan, 2., np.nan, np.nan, 5.],
              index=['a', 'b', 'c', 'd', 'e', 'f'])

In [97]:
a

f    NaN
e    2.5
d    0.0
c    3.5
b    4.5
a    NaN
dtype: float64

In [98]:
b

a    0.0
b    NaN
c    2.0
d    NaN
e    NaN
f    5.0
dtype: float64

In [99]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 0. , 3.5, 4.5, 5. ])

Uma Series tem um método *combine_first*, que executa o equivalente
a essa operação, junto com a lógica usual de alinhamento de dados
do pandas:

In [100]:
b.combine_first(a)

a    0.0
b    4.5
c    2.0
d    0.0
e    2.5
f    5.0
dtype: float64

Com DataFrames, *combine_first* faz o mesmo coluna a coluna,
portanto podemos pensar nele como se estivesse fazendo um
“patching” dos dados ausentes no objeto que faz a chamada, com
os dados do objeto que você lhe passar:

In [102]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})

df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})

In [103]:
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [104]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [105]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


### Reformatação e pivoteamento

Há uma série de operações básicas para reorganização de dados
tabulares. De modo alternativo, são chamadas de operações de
*reformatação* (reshaping) ou de *pivoteamento* (pivoting).

### Reformatação com indexação hierárquica

A indexação hierárquica oferece uma forma consistente de
reorganizar dados em um DataFrame. Há duas ações principais:

- *stack*
  - Faz a “rotação” ou o pivoteamento das colunas dos dados para as linhas.
- *unstack*
  - Faz o pivoteamento das linhas para as colunas.
  
Demonstrarei essas operações com uma série de exemplos.
Considere um pequeno DataFrame com arrays de strings como
índices das linhas e das colunas:

In [106]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'],
                    name='number'))

In [107]:
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


Utilizar o método *stack* nesses dados faz o pivoteamento das colunas
para as linhas, gerando uma Series:

In [108]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

A partir de uma Series indexada hierarquicamente, podemos
reorganizar os dados de volta em um DataFrame usando *unstack*:

In [109]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


Por padrão, o nível mais interno será desempilhado (o mesmo vale
para *stack*). Podemos desempilhar um nível diferente passando um
número ou o nome de um nível:

In [110]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [111]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [112]:
result.unstack('number')

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


Desempilhar pode introduzir dados ausentes se nem todos os
valores do nível forem encontrados em cada um dos subgrupos:

In [113]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])

In [114]:
data2 = pd.concat([s1, s2], keys=['one', 'two'])

In [115]:
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [116]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


O empilhamento filtra dados ausentes por padrão, de modo que a
operação seja mais facilmente revertida:

In [117]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [118]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

Quando desempilhamos dados em um DataFrame, o nível
desempilhado passa a ser o nível mais baixo no resultado:

In [119]:
df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns=pd.Index(['left', 'right'], name='side'))

In [120]:
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [121]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

Quando chamamos *stack*, podemos informar o nome do eixo a ser
empilhado:

In [122]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

### Fazendo o pivoteamento de um formato “longo” para um formato “largo”

Uma maneira comum de armazenar várias séries temporais em
bancos de dados e em CSV é usar o chamado formato *longo* (long)
ou *empilhado* (stacked). Vamos carregar alguns dados de exemplo efazer uma pequena manipulação de séries temporais e outras
operações de limpeza de dados:

In [132]:
path = '/content/drive/MyDrive/Análise de Dados/CSVs e TXTs/macrodata.csv'
data = pd.read_csv(path, delimiter=';')

In [133]:
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [135]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter, name='date')
periods

PeriodIndex(['1959Q1', '1959Q2', '1959Q3', '1959Q4', '1960Q1'], dtype='period[Q-DEC]', name='date', freq='Q-DEC')

In [137]:
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
columns

Index(['realgdp', 'infl', 'unemp'], dtype='object', name='item')

In [138]:
data = data.reindex(columns=columns)
data

item,realgdp,infl,unemp
0,2710.349,0.00,5.8
1,2778.801,2.34,5.1
2,2775.488,2.74,5.3
3,2785.204,0.27,5.6
4,2847.699,2.31,5.2


In [139]:
data.index = periods.to_timestamp('D', 'end')
data

item,realgdp,infl,unemp
date,,,
1959-03-31 23:59:59.999999999,2710.349,0.00,5.8
1959-06-30 23:59:59.999999999,2778.801,2.34,5.1
1959-09-30 23:59:59.999999999,2775.488,2.74,5.3
1959-12-31 23:59:59.999999999,2785.204,0.27,5.6
1960-03-31 23:59:59.999999999,2847.699,2.31,5.2


In [140]:
Idata = data.stack().reset_index().rename(columns={0: 'value'})

Para resumir, ele combina as colunas *year* e *quarter* de modo a criar
um tipo de intervalo de tempo.

Eis a aparência de *ldata*:

In [141]:
Idata[:10]

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


Esse é o chamado formato *longo* para várias séries temporais, ou
outros dados observados com duas ou mais chaves (nesse caso,
nossas chaves são a data e o item). Cada linha da tabela representa
uma única observação.

Os dados são frequentemente armazenados dessa maneira em
bancos de dados relacionais como o MySQL, pois um esquema fixo
(nomes de coluna e tipos de dados) permite que o número de
valores distintos na coluna item mude à medida que dados são
adicionados à tabela. No exemplo anterior, *date* e *item* normalmente
seriam as chaves primárias (no jargão de bancos de dados
relacionais), oferecendo tanto uma integridade relacional quanto
junções mais fáceis. Em alguns casos, talvez seja mais difícil
trabalhar com os dados nesse formato; você pode preferir ter um
DataFrame contendo uma coluna por valor distinto de *item* indexado
por timestamps na coluna *date*. O método *pivot* de DataFrame
executa exatamente essa transformação:

In [142]:
pivoted = Idata.pivot('date', 'item', 'value')
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


Os dois primeiros valores especificados são as colunas a serem
usadas respectivamente como o índice das linhas e das colunas e,
então, por fim, um valor de coluna opcional para preencher o
DataFrame. Suponha que tivéssemos dois valores de coluna que
quiséssemos reformatar simultaneamente:

In [149]:
size = len(Idata)
Idata['value2'] = np.random.randn(size)

In [150]:
Idata[: 10]

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,0.619154
1,1959-03-31 23:59:59.999999999,infl,0.000,0.134024
2,1959-03-31 23:59:59.999999999,unemp,5.800,0.785751
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,1.369420
4,1959-06-30 23:59:59.999999999,infl,2.340,1.394835
5,1959-06-30 23:59:59.999999999,unemp,5.100,-0.323907
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,-0.894759
7,1959-09-30 23:59:59.999999999,infl,2.740,0.088830
8,1959-09-30 23:59:59.999999999,unemp,5.300,-0.468285
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,0.918163


Ao omitir o último argumento, obteremos um DataFrame com
colunas hierárquicas:

In [151]:
pivoted = Idata.pivot('date', 'item')
pivoted[:5]

value            ...    value2          
item                           infl   realgdp  ...   realgdp     unemp
date                                           ...                    
1959-03-31 23:59:59.999999999  0.00  2710.349  ...  0.619154  0.785751
1959-06-30 23:59:59.999999999  2.34  2778.801  ...  1.369420 -0.323907
1959-09-30 23:59:59.999999999  2.74  2775.488  ... -0.894759 -0.468285
1959-12-31 23:59:59.999999999  0.27  2785.204  ...  0.918163 -0.398305
1960-03-31 23:59:59.999999999  2.31  2847.699  ... -0.652355  1.229458

[5 rows x 6 columns]

In [152]:
pivoted['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


Observe que *pivot* é equivalente a criar um índice hierárquico usando
*set_index* seguido de uma chamada a *unstack*:

In [153]:
unstacked = Idata.set_index(['date', 'item']).unstack('item')
unstacked[:7]

value            ...    value2          
item                           infl   realgdp  ...   realgdp     unemp
date                                           ...                    
1959-03-31 23:59:59.999999999  0.00  2710.349  ...  0.619154  0.785751
1959-06-30 23:59:59.999999999  2.34  2778.801  ...  1.369420 -0.323907
1959-09-30 23:59:59.999999999  2.74  2775.488  ... -0.894759 -0.468285
1959-12-31 23:59:59.999999999  0.27  2785.204  ...  0.918163 -0.398305
1960-03-31 23:59:59.999999999  2.31  2847.699  ... -0.652355  1.229458

[5 rows x 6 columns]

### Pivoteamento do formato “largo” para o formato “longo”

Uma operação inversa de *pivot* para DataFrames é *pandas.melt*. Em
vez de transformar uma coluna em várias em um novo DataFrame,
ele fará o merge de várias colunas em uma só, gerando um
DataFrame maior que a entrada. Vamos ver um exemplo:

In [154]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


A coluna *'key'* pode ser um indicador de grupo, e as outras colunas
são valores de dados. Ao usar *pandas.melt*, devemos informar quais
colunas (se houver) são indicadores de grupo. Vamos usar *'key'*
como o único indicador de grupo nesse caso:

In [156]:
melted = pd.melt(df, ['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


Usando *pivot*, podemos reformatar novamente e obter o layout
original:

In [157]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


Como o resultado de *pivot* cria um índice a partir da coluna usada
como os rótulos das linhas, podemos usar *reset_index* para passar os
dados de volta para uma coluna:

In [158]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


Também é possível especificar um subconjunto de colunas a serem
usadas como colunas de valores:

In [159]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


*pandas.melt* também pode ser usado sem nenhum identificador de
grupo:

In [160]:
pd.melt(df, value_vars=['A', 'B', 'C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [161]:
pd.melt(df, value_vars=['key', 'A', 'B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6
